Import data reader

In [73]:
from get_test_input import read_input
import re

### Day 1

In [2]:
lines = read_input(day=1, round=1, test=False)
left_list, right_list = [int(l.split('   ')[0]) for l in lines], [int(l.split('   ')[1]) for l in lines]
left_list.sort()
right_list.sort()
print('Day 1 round 1 answer =', sum([abs(x-y) for x,y in zip(left_list, right_list)]))
print('Day 1 round 2 answer =', sum([x*right_list.count(x) for x in list(set(left_list).intersection(set(right_list))) ]))

Day 1 round 1 answer = 2164381
Day 1 round 2 answer = 20719933


### Day 2

In [52]:
lines = read_input(day=2, round=1, test=False)
reports = [[int(x) for x in l.split(' ')] for l in lines]

def is_safe(report):
   return abs(sum([(report[i] - report[i+1])/abs(report[i] - report[i+1]) if 0 < abs(report[i] - report[i+1]) <= 3  else 0 for i in range(len(report)-1 )]))==len(report)-1

safe_reports = sum([is_safe(report) for report in reports])

def is_safe_with_damping(report):
    return is_safe(report) or any(is_safe(report[:i] + report[i + 1:]) for i in range(len(report)))

safe_reports_with_damping = sum([is_safe_with_damping(report) for report in reports])

print('Day 2 round 1 answer =', safe_reports)
print('Day 2 round 2 answer =', safe_reports_with_damping)

Day 2 round 1 answer = 306
Day 2 round 2 answer = 366


### Day 3

In [49]:
multipliers = read_input(day=3, round=1, test=True,regex_screen=r'mul\(\d{1,3},\d{1,3}\)')
total_1 = sum([int(mul[mul.index('(')+1:mul.index(',')])*int(mul[mul.index(',')+1:-1]) for mul in multipliers])
multipliers_and_enablers = read_input(day=3, round=1, test=False,regex_screen=r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)')

skip = False
enabled_multipliers = list(filter(lambda x: x!='do()',[mul_enabler for mul_enabler in multipliers_and_enablers if  skip == False and mul_enabler != "do()" and  not (skip := mul_enabler == "don't()") or (mul_enabler == "do()" and not (skip := False))]))
total_2 = sum([int(mul[mul.index('(')+1:mul.index(',')])*int(mul[mul.index(',')+1:-1]) for mul in enabled_multipliers])
 
print('Day 3 round 1 answer =', total_1)
print('Day 3 round 2 answer =', total_2)

Day 3 round 1 answer = 161
Day 3 round 2 answer = 72948684


### Day 4

In [99]:
grid = read_input(day=4, test=False)
def get_rows_columns_diagonals(grid):
    rows , columns = grid , [''.join(col) for col in zip(*grid)]
    num_rows, num_cols= len(columns), len(rows)

    diagonals = [''.join([grid[r + d][d] for d in range(min(num_rows - r, num_cols))]) for r in range(num_rows)] + [
                ''.join([grid[d][c + d] for d in range(min(num_rows, num_cols - c))]) for c in range(1, num_cols)] +[
                ''.join([grid[r + d][num_cols - 1 - d] for d in range(min(num_rows - r, num_cols))]) for r in range(num_rows)] + [
                ''.join([grid[d][c - d] for d in range(min(num_rows, c + 1))])for c in range(num_cols - 1)]
    return rows + columns + diagonals

def get_x_s(grid):
    return [(grid[r+1][c+1]+grid[r][c]+grid[r-1][c-1],grid[r-1][c+1]+grid[r][c]+grid[r+1][c-1]) for r in range(1, len(grid)-1) for c in range(1, len(grid[0])-1)]

def count_string_appearances(strings, substring):
    return sum([len(re.findall(f'(?={re.escape(substring)})', string))+len(re.findall(f'(?={re.escape(substring[::-1])})', string)) for string in strings])

def count_x_strings(xs, string):
    return sum([string_1 in (string ,string[::-1]) and string_2 in (string ,string[::-1]) for string_1, string_2 in xs])

round_1 = count_string_appearances(get_rows_columns_diagonals(grid), 'XMAS')
round_2 = count_x_strings(get_x_s(grid), 'MAS')

print('Day 4 round 1 answer =', round_1)
print('Day 4 round 2 answer =', round_2)

Day 4 round 1 answer = 2545
Day 4 round 2 answer = 1886


### Day 5

In [229]:
#code to read input
rules, updates = read_input(day=5,test=False,input_delimter='\n\n')

rules = rules.split('\n')
updates = [update.split(',') for update in updates.split('\n')]

# Define rule dictionary - originally did a before and after dict, is that more efficient? don't think so
rule_dict = {}
for rule in rules:
    before , after = rule.split('|')
    if after in rule_dict:
        rule_dict[after].add(before)
    else:
        rule_dict[after] ={before}


# If return correction = True we are answering part two and we will fix the update and then return the new middle entry
def validate_update(rule_dict, original_update, return_correction = False):
    update = original_update.copy()
    i = 0
    while i < len(update):

        page , afters,  befores = update[i], set(update[i+1:]), set(update[:i])
        should_be_before={}
        if (page in rule_dict and (should_be_before := afters & rule_dict[page])):
            
            if not return_correction:
                return False
            else:
                update = update[:i] +[a for a in update[i+1:] if a in should_be_before ]+ [a for a in update[i:] if  a not in should_be_before ]
                i -= 1 
        i += 1
    
    else:
        if not return_correction:
            return  True
        elif update != original_update:
            return int(update[i//2])
        else:
            return 0

# Add up results for answers to part 1 and 2
round_1 = sum([int(update[len(update)//2 ]) for update in updates if validate_update(rule_dict,update)])
round_2 = sum([validate_update(rule_dict, update, return_correction=True) for update in updates])
print('Day 5 round 1 answer =', round_1)
print('Day 5 round 2 answer =', round_2)


Day 5 round 1 answer = 5374
Day 5 round 2 answer = 4260
